In [1]:
import sys
sys.path.append(r"C:\Users\Felix\Documents\.Uni\Project\Git\tum-ase-project-2025-1_2\optimization")

import numpy as np
import pandas as pd
import plotly.express as px
from trained_surrogate import predict_rf_and_weight

# --- Your base input vector (25-dimensional) ---
base_vector = [
    28.02919603, 28.85779898, 28.44662807, 27.43628104, 28.66617122,
    27.73155523, 24.64874037, 22.95335333, 26.22720405, 24.21180338,
    6.05329586, 6.49089979, 6.159172323, 5.418504035, 6.144673094,
    4.442702595, 4.819598924, 5.202283445, 5.247025691, 4.421178302,
    4.735314264, 4.895437801, 4.935854921, 4.856812295, 5.590097472
]

# --- Define percent deviation (+/- as a fraction, e.g., 0.06 for 6%) ---
deviation = 0.01  # 6%

# --- Number of samples to generate ---
n_samples = 2000


# --- Calculate per-variable bounds ---
base = np.array(base_vector)
lower_bounds = base * (1 - deviation)
upper_bounds = base * (1 + deviation)

# Optional: enforce min/max if you wish to not go out of physical/realistic range
#abs_min = np.array([20]*5 + [20]*5 + [4.5]*5 + [4.2]*5 + [4.6]*5)
#abs_max = np.array([30]*5 + [30]*5 + [6.5]*5 + [5.6]*5 + [5.7]*5)
#lower_bounds = np.maximum(lower_bounds, abs_min)
#upper_bounds = np.minimum(upper_bounds, abs_max)

# --- Generate random samples in this local box ---
random_vectors = np.random.uniform(lower_bounds, upper_bounds, size=(n_samples, 25))

results = []
for x in random_vectors:
    rf_est, weight_est = predict_rf_and_weight(x)
    if 1.00 <= rf_est <= 1.05 and weight_est < 28.5:
        results.append(list(x) + [rf_est, weight_est])

# --- Save results to CSV ---
columns = [f'input_{i+1}' for i in range(25)] + ['predicted_min_nonstrength_rf', 'predicted_weight']
df_results = pd.DataFrame(results, columns=columns)
df_results.to_csv('surrogate_local_candidates.csv', index=False)
print(f"Saved {len(df_results)} samples in local region to 'surrogate_local_candidates.csv'.")

# --- Plot the results ---
fig = px.scatter(
    df_results, x='predicted_weight', y='predicted_min_nonstrength_rf',
    labels={'predicted_weight': 'Predicted Weight (kg)', 'predicted_min_nonstrength_rf': 'Predicted min_nonstrength_rf'},
    title=f"Surrogate Model: Local Scan (+/-{int(deviation*100)}%) Region (RF 1.00–1.05, Weight < 28.5 kg)"
)
fig.show()


Saved 2000 samples in local region to 'surrogate_local_candidates.csv'.
